## Syntax-dependent trace



In [12]:
!pip install huggingface_hub

In [3]:
!pip install git+https://github.com/davidbau/baukit

  Cloning https://github.com/davidbau/baukit to /tmp/pip-req-build-mxs6edvd
  Running command git clone -q https://github.com/davidbau/baukit /tmp/pip-req-build-mxs6edvd
  Resolved https://github.com/davidbau/baukit to commit e14a18a6ad6cf9e0d6a5dc7a97e671e393c01682
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for baukit: filename=baukit-0.0.1-py3-none-any.whl size=59669 sha256=6c220dcc44e3955e13c659ab88012ceef3b2218fa703853ddf5aff0d8a68348d
  Stored in directory: /tmp/pip-ephem-wheel-cache-6smm7l4o/wheels/7c/44/0a/28ca5746c56670d449cbe468868a8081e8f5a31b01604b3929
Successfully built baukit


In [1]:
import sys
sys.path.append('..')
import tqdm as notebook_tqdm
from model import ModelLoader
from huggingface_hub import hf_hub_download
import gc
import torch

CACHE_DIR = "/scratch/lucchetti.f/models/"

/home/lucchetti.f/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def check_dev(n):
    t = torch.cuda.get_device_properties(n).total_memory
    r = torch.cuda.memory_reserved(n)
    a = torch.cuda.memory_allocated(n)
    f = r-a  # free
    print(f"{a} / {t} used for device {n}, reserved {r}")
    
def check_devs():
    for i in range(torch.cuda.device_count()):
        check_dev(i)

check_devs()

0 / 11997020160 used for device 0, reserved 0


In [3]:
MODEL_NAME = "Salesforce/codegen-16B-mono"
## Optional Download the Weights
# weights_location = hf_hub_download(MODEL_NAME, "pytorch_model.bin", cache_dir=CACHE_DIR)
# hf_hub_download(MODEL_NAME, "config.json", cache_dir=CACHE_DIR)

In [4]:
loaded_model = ModelLoader(MODEL_NAME)

/scratch/lucchetti.f/models/models--Salesforce--codegen-16B-mono


OSError: /scratch/lucchetti.f/models/models--Salesforce--codegen-16B-mono does not appear to have a file named config.json. Checkout 'https://huggingface.co//scratch/lucchetti.f/models/models--Salesforce--codegen-16B-mono/None' for available files.

In [5]:
check_devs()

0 / 11997020160 used for device 0, reserved 0


## Inference

In [ ]:
model = loaded_model.model
tokenizer = loaded_model.tokenizer

In [ ]:
prompt = "<html><!-- A  HTML page for a search engine, with a simple text box in the center -->",
    stop_sequences=["</html>"])
input = tokenizer.tokenize(prompt)
input.to(0)
codegen.generate(input)

## Trace + activation patch